## Train test + recommender
* Todo : add embeddings for items/users (proteins) - dense (embeddings), sparse (features)
* Libraries:
    * lightFM - https://github.com/lyst/lightfm
        * example loading metadata/side data (and transforming it into dict) - https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/
            * https://stackoverflow.com/questions/49896816/how-do-i-optimize-the-hyperparameters-of-lightfm
    * Surprise
    *( Spotlight (deep pytorch) - https://github.com/maciejkula/spotlight
        * partial metadata support - https://github.com/maciejkula/spotlight/issues/79
    * Keras Deep CTR
    
    
* Example post about implicit recomendation & using user/item metadata - https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/

* Hyperparameter tuning + compariosn of lightFM to others (with item metadata included + SVD) : https://www.eigentheories.com/blog/lightfm-vs-hybridsvd/

In [1]:
import pandas as pd
from lightfm import LightFM
import os
from scipy.sparse import coo_matrix
import numpy as np

from lightfm.evaluation import precision_at_k , auc_score
from scipy.sparse import vstack

C:\Users\Dan Ofer\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
def get_coo(df,index_labels =2):
    labels = df.iloc[:, index_labels:]
    sparse_label_matrix = coo_matrix(labels.values)
    return(sparse_label_matrix)


### hyperparam optimization for lightfm 

# from skopt import forest_minimize
# def objective(params):
#     # unpack
#     epochs, learning_rate,\
#     no_components, alpha = params
    
#     user_alpha = alpha
#     item_alpha = alpha
#     model = LightFM(loss='warp',
#                     random_state=2016,
#                     learning_rate=learning_rate,
#                     no_components=no_components,
#                     user_alpha=user_alpha,
#                     item_alpha=item_alpha)
#     model.fit(train, epochs=epochs,
#               num_threads=4, verbose=True)
    
#     patks = lightfm.evaluation.precision_at_k(model, test,
#                                               train_interactions=None,
#                                               k=5, num_threads=4)
#     mapatk = np.mean(patks)
#     # Make negative because we want to _minimize_ objective
#     out = -mapatk
#     # Handle some weird numerical shit going on
#     if np.abs(out + 1) < 0.01 or out < -1.0:
#         return 0.0
#     else:
#         return out

# space = [(1, 260), # epochs
#          (10**-4, 1.0, 'log-uniform'), # learning_rate
#          (20, 200), # no_components
#          (10**-6, 10**-1, 'log-uniform'), # alpha
#         ]

# res_fm = forest_minimize(objective, space, n_calls=20,
#                      random_state=0,
#                      verbose=True)

In [3]:
DATA_DIR = r"C:\Users\Dan Ofer\Desktop\Stuff\cafa\ProteinBert\data\training_data"

### toy seqs - few labels ,smaller
# TRAIN_FILE_LOC = os.path.join(DATA_DIR,"labelled_toy_seqs_v1_TRAIN.csv")
# TEST_FILE_LOC = os.path.join(DATA_DIR,"labelled_toy_seqs_v1_TEST.csv")


TRAIN_FILE_LOC = os.path.join(DATA_DIR,r"SWP_test_go-all_v1.csv.gz")
TEST_FILE_LOC = os.path.join(DATA_DIR,r"SWP_train_go-all_v1.csv.gz")

### load our data and get it into sparse format
* start with just the test -  faster
* skip embeddings data for now

In [4]:
df = pd.read_csv(TRAIN_FILE_LOC,nrows=5).reset_index()
print(df.shape)
print(df.columns)
df.head()

(5, 126)
Index(['index', 'Sequence', ' 4 iron, 4 sulfur cluster binding [GO:0051539]',
       ' ATP binding [GO:0005524]', ' ATPase activity [GO:0016887]',
       ' ATPase-coupled transmembrane transporter activity [GO:0042626]',
       ' DNA binding [GO:0003677]', ' DNA recombination [GO:0006310]',
       ' DNA repair [GO:0006281]', ' DNA replication [GO:0006260]',
       ...
       'endoplasmic reticulum membrane [GO:0005789]',
       'extracellular region [GO:0005576]', 'extracellular space [GO:0005615]',
       'integral component of membrane [GO:0016021]',
       'integral component of plasma membrane [GO:0005887]',
       'mitochondrion [GO:0005739]', 'nucleolus [GO:0005730]',
       'nucleus [GO:0005634]', 'plasma membrane [GO:0005886]',
       'ribosome [GO:0005840]'],
      dtype='object', length=126)


,index,Sequence,"4 iron, 4 sulfur cluster binding [GO:0051539]",ATP binding [GO:0005524],ATPase activity [GO:0016887],ATPase-coupled transmembrane transporter activity [GO:0042626],DNA binding [GO:0003677],DNA recombination [GO:0006310],DNA repair [GO:0006281],DNA replication [GO:0006260],...,endoplasmic reticulum membrane [GO:0005789],extracellular region [GO:0005576],extracellular space [GO:0005615],integral component of membrane [GO:0016021],integral component of plasma membrane [GO:0005887],mitochondrion [GO:0005739],nucleolus [GO:0005730],nucleus [GO:0005634],plasma membrane [GO:0005886],ribosome [GO:0005840]
0,0,MNLSQEHAINQNLHKNQKNEEKIEKKTINKDGRGQMNYDGEEGQGE...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,MDVDVFNGWGRPRFEDESLMPPGFRFHPTDEELITYYLLKKVLDSN...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2,MDKKKEHPEMRIPLQTAVEVSDWPCSTSHDPHSGLGMVLGMLAVLG...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,MARASLIQPGLWALLLLQAVGPAVAAKLNIPKVLLPFTRATRVNFT...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,MEESDSEKTTEKENLGPRMDPPLGEPEGSLGWVLPNTAMKKKVLLM...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### OPT - get only sequences with at least 2 annotations
* rememebr that we'll need to address this when predicting on our real test set

In [5]:
df = pd.concat([pd.read_csv(TRAIN_FILE_LOC),pd.read_csv(TEST_FILE_LOC)])
print("all",df.shape[0])
print("at least 1 annotations",df.loc[df.sum(axis=1)>=1].shape[0])
print("at least 2 annotations",df.loc[df.sum(axis=1)>=2].shape[0])
print("at least 3 annotations",df.loc[df.sum(axis=1)>=3].shape[0])

df.head(1)

all 774746
at least 1 annotations 652138
at least 2 annotations 433273
at least 3 annotations 252031


,Sequence,"4 iron, 4 sulfur cluster binding [GO:0051539]",ATP binding [GO:0005524],ATPase activity [GO:0016887],ATPase-coupled transmembrane transporter activity [GO:0042626],DNA binding [GO:0003677],DNA recombination [GO:0006310],DNA repair [GO:0006281],DNA replication [GO:0006260],DNA-binding transcription factor activity [GO:0003700],...,endoplasmic reticulum membrane [GO:0005789],extracellular region [GO:0005576],extracellular space [GO:0005615],integral component of membrane [GO:0016021],integral component of plasma membrane [GO:0005887],mitochondrion [GO:0005739],nucleolus [GO:0005730],nucleus [GO:0005634],plasma membrane [GO:0005886],ribosome [GO:0005840]
0,MNLSQEHAINQNLHKNQKNEEKIEKKTINKDGRGQMNYDGEEGQGE...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
### keep only sequencers with at least 1 annotation
df = df.loc[df.sum(axis=1)>=1]

In [7]:
# df = df.replace(0,-1) ###  matters for lightfm , notably logistic loss ??

In [8]:
# data1 = get_coo(pd.read_csv(TRAIN_FILE_LOC).reset_index(),index_labels =2)
# data2 = get_coo(pd.read_csv(TEST_FILE_LOC).reset_index(),index_labels =2)
# data = vstack([data1, data2])

data = get_coo(df,index_labels =1)

In [9]:
from scipy.sparse import find

TEST_FREQ = 0.1

rows, cols, values = find(data)

test_mask = np.random.choice([True, False], len(rows), p = [TEST_FREQ, 1- TEST_FREQ])
train_mask = ~test_mask

train = coo_matrix((values[train_mask], (rows[train_mask], cols[train_mask])), shape = data.shape)
test = coo_matrix((values[test_mask], (rows[test_mask], cols[test_mask])), shape = data.shape)

print(train.shape, test.shape)

(652138, 124) (652138, 124)


In [10]:
model = LightFM(learning_rate=0.05,
#                 loss='logistic', ## logistic is super slow ; changes if we replace 0 with -1 
                loss='warp'
               )
model.fit(train, epochs=5)

In [11]:
train_precision = precision_at_k(model, train, k=5).mean()
test_precision = precision_at_k(model, test, k=5, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.35, test 0.10.
AUC: train 0.98, test 0.87.


In [12]:
precision_at_k(model, test, k=1, train_interactions=train).mean()

0.2624587

In [13]:
precision_at_k(model, test, k=2, train_interactions=train).mean()

0.18150276

In [14]:
precision_at_k(model, test, k=3, train_interactions=train).mean()

0.14337859

5 epochs ; All data

* Basic  - 
    * WARP
        * Precision: train 0.34, test 0.11.

            AUC: train 0.98, test 0.86.
    * BPR -
        * Precision: train 0.29, test 0.07.
        
            AUC: train 0.95, test 0.70.
    
    * Logistic loss (much worse):
        * Precision: train 0.11, test 0.06.
            AUC: train 0.77, test 0.76.
      

* With tfidf "metadata"  - 
    * WARP
        * Precision: train 0.19, test 0.11.
            AUC: train 0.88, test 0.88.
            
---------------

At Least 2 labels ; 5 epochs

* Basic  - 
    * WARP
        *  Precision: train 0.43, test 0.12.
        
            AUC: train 0.98, test 0.88.
---------------

At Least 1 label  ; replace 0 with -1 ; 5 epochs  (652,138)
    * Logistic loss
        * 

## Add sequence/"user" metadata
* get vector of item features - start with TF-IDF/count/char-ngrams vector per sequence
    * Could also add the uniprot (sparse) "features" 
    
__Q__ : Are we sure order is saved ??  

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.pipeline import Pipeline, FeatureUnion

from lightfm import datasets as dataset
import scipy.sparse as sp

from sklearn.preprocessing import FunctionTransformer
### https://stackoverflow.com/questions/47745288/how-to-featureunion-numerical-and-text-features-in-python-sklearn-properly

### https://michelleful.github.io/code-blog/2015/06/20/pipelines/
##### could also add sequence length , PH for example

In [16]:
seqs = pd.concat([pd.read_csv(TRAIN_FILE_LOC),pd.read_csv(TEST_FILE_LOC)])["Sequence"]
print(len(seqs))
# seqs

774746


In [17]:
char_grams = TfidfVectorizer(ngram_range=(1, 3), analyzer='char',min_df=5,
                             lowercase=False, max_df=0.997,
                             max_features=4000 , smooth_idf =False)
svd = TruncatedSVD(n_components=25)

In [18]:
item_feats = svd.fit_transform(char_grams.fit_transform(seqs))

# item_feats = char_grams.fit_transform(seqs)

In [19]:
# item_features = dataset.build_item_features(features)

item_features_sparse = sp.coo_matrix(item_feats)
print(item_features_sparse.shape)
item_features_sparse

(774746, 25)


<774746x25 sparse matrix of type '<class 'numpy.float64'>'
	with 19368650 stored elements in COOrdinate format>

In [20]:
model2 = LightFM(learning_rate=0.05, loss='warp')

In [21]:
model2.fit(train,user_features=item_features_sparse, epochs=5)

# model2.fit(train,item_features = sp.coo_matrix(item_feats), epochs=1)

In [22]:
train_precision = precision_at_k(model2, train, k=5,user_features=item_features_sparse).mean()
test_precision = precision_at_k(model2, test, k=5, train_interactions=train,user_features=item_features_sparse).mean()

train_auc = auc_score(model2, train,user_features=item_features_sparse).mean()
test_auc = auc_score(model2, test, train_interactions=train,user_features=item_features_sparse).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

# Precision: train 0.53, test 0.24.
# AUC: train 0.84, test 0.87.

Precision: train 0.12, test 0.06.
AUC: train 0.77, test 0.76.
